In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
import seaborn as sns
import zipfile

In [ ]:
plt.rcParams["axes.labelsize"] = 14
plt.rcParams["axes.titlesize"] = 18

# Loading dataframes

In [ ]:
VOTE_DATA_PATH = 'data/votes/'
Z_FILE = zipfile.ZipFile(VOTE_DATA_PATH+'votes.zip')

vote_data_files = [x.filename for x in Z_FILE.infolist() if x.filename.startswith('votes_')]

In [ ]:
def interpret_position(positions):
    r = positions.copy()
    r = r.apply(lambda x : 1 if x == "Yes" else ( -1 if x == "No" else 0 ))
    
    return r

In [ ]:
USEFUL_COLUMNS = ['id','bill.number','congress','member_id','question','position', 'timestamp']

def process_vote_file(file_path):
    print("Processing vote data for {file}".format(file=file_path))
    ifile = Z_FILE.open(file_path)
    
    # Read data
    df = pd.read_csv(ifile)

    # We drop entries without bill number as they lead to invalid bill IDs
    df = df[df['bill.number'].notna()]
    
    # We drop entries without roll call value
    df = df[df['roll_call'].notna()]
    
    # As there can be several votes per bill we need to build a unique ID
    df['id'] = df['bill.bill_id'].map(str) + "-" + df['roll_call'].map(str)

    # Some bill IDs and roll call numbers are poorly filled, leading to duplicates
    df = df.drop_duplicates('id')
    
    # Build timestamps
    df['timestamp'] = pd.to_datetime(df.date.map(str) + " " + df.time.map(str))
    
    # Keep only useful columns
    df = df[USEFUL_COLUMNS]
    
    # Convert position to numeric
    df['position'] = interpret_position(df['position'])
    
    return df

In [ ]:
NOMINATION_VOTE = "On the Nomination"
MOTION_VOTE = "On the Motion"
CLOTURE_MOTION_VOTE = "On the Cloture Motion"
BILL_PASSAGE = "On Passage of the Bill"

def build_vote_matrix(questions, data):
    """
    Given a list of questions, build a vote matrix.
    Index are senators IDs, columns are vote IDs and values are the respective positions
    of the senators with respect to the question (Yes, No, blank or NaN if the senator
    did not take part in the vote)
    
    Data should contain the following columns:
        - senators ID (member_id)
        - vote ID (id)
        - position (position)
    """
    vote_matrix = data[data['question'].isin(questions)]
    vote_matrix = vote_matrix.pivot(columns='id', index='member_id', values='position')
    
    return vote_matrix

In [ ]:
# Example vote matrix
vm_data1 = process_vote_file(vote_data_files[210])
vm_data2 = process_vote_file(vote_data_files[202])
vm_data = pd.concat([vm_data1.head(10), vm_data2.head(10)]) 

display(vm_data)

vm = build_vote_matrix([NOMINATION_VOTE,CLOTURE_MOTION_VOTE,MOTION_VOTE], vm_data)

display(vm)

In [ ]:
# Compiling senators from all congresses
raw_senators = pd.DataFrame()

for i in range (115, 80 - 1, -1):
    df = pd.read_csv("data/senate_members/senate_members_{congress}.csv".format(congress = i))
    df['congress'] = i
    raw_senators = pd.concat([raw_senators, df], sort=False)
    
    
# Compiling commities
raw_commities = pd.DataFrame()

for i in range (115, 110 - 1, -1):
    df = pd.read_csv("data/commities/commities_{congress}.csv".format(congress = i))
    df['congress'] = i
    raw_commities = pd.concat([raw_senators, df], sort=False)
    
    
# Retrieving votes
raw_votes = pd.read_csv(Z_FILE.open("votes.csv"))

#Retrieving lobbies
raw_lobbies = pd.read_csv("data/lobby/lobby.csv")

In [ ]:
raw_senators.columns

In [ ]:
# Do senators tend to vote more and more with their party ?

data = raw_senators[raw_senators['congress'] > 100]
ax = data.boxplot(column='votes_with_party_pct', by='congress', figsize=(15,10))
ax.set(xlabel="Congress", ylabel="Percentage of votes with party")
ax.get_figure().suptitle("")

plt.title("Evolution of voting with party percentage")
plt.show()

In [ ]:
gc = gender_count.unstack()
gc.columns = gc.columns.droplevel()
# Compute gender percentages
gc = gc.div(gc.sum(axis=1),axis=0).fillna(0)
gc['F'].plot(kind='line', figsize=(15,10), marker='o', linestyle='-')
plt.title("Proportion of women in senate")

In [ ]:
raw_votes.columns

In [ ]:
raw_votes.head()